In [373]:
import datetime as dt
from datetime import date
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn import preprocessing

# Data Input, EDA & Preprocessing

## Individual Data Input

In [374]:
price_vol = pd.read_csv('../data/Price & Volume BTC.csv')
tweets = pd.read_csv('../data/Tweets.csv')
funding_rates_1600 = pd.read_csv('../data/Funding Rates BTCUSDT 1600.csv')
funding_rates_0800 = pd.read_csv('../data/Funding Rates BTCUSDT 0800.csv')
funding_rates_0000 = pd.read_csv('../data/Funding Rates BTCUSDT 0000.csv')
google = pd.read_csv('../data/Google.csv')
transactions = pd.read_csv('../data/Transactions.csv')
unique_addresses = pd.read_csv('../data/Unique Addresses.csv')
active_addresses = pd.read_csv('../data/Active Addresses.csv')

## Data Compilation

In [375]:
# get daily ave value for twitter attributes
tweets = tweets.groupby(by="Time").mean()
tweets = tweets.reset_index()

df = pd.merge(price_vol, funding_rates_0000, on=['Time'])
df = pd.merge(df, funding_rates_0800, on=['Time'])
df = pd.merge(df, funding_rates_1600, on=['Time'])
df = pd.merge(df, google, on=['Time'])
df = pd.merge(df, active_addresses, on=['Time'])
df = pd.merge(df, unique_addresses, on=['Time'])
df = pd.merge(df, transactions, on=['Time'])
df = pd.merge(tweets, df, how="outer")
df.head()

,Time,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,Close,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,2020-03-12,5.0,2.0,0.0772,0.091,0.100,0.809,7933.2,4802.9,7960.4,4448.6,55248355.79,0.02%,0.01%,0.01%,22.83,6.20,909601,653156.0,349978.0
1,2020-03-17,98.0,96.0,0.4019,0.000,0.114,0.886,5028.8,5316.7,5532.0,4926.1,18269210.08,-0.09%,-0.02%,0.00%,10.21,3.39,765477,540902.0,278326.0
2,2020-03-20,125.0,95.0,0.4767,0.000,0.146,0.854,6162.2,6204.4,6901.7,5661.9,37752569.82,-0.03%,-0.02%,0.00%,10.04,3.56,734614,564748.0,284594.0
3,2020-03-30,3.0,0.0,0.0803,0.061,0.071,0.869,5881.3,6394.9,6600.0,5860.0,17904828.40,-0.03%,-0.06%,-0.03%,5.92,3.63,770915,535787.0,276828.0
4,2020-03-31,186.0,142.0,0.6597,0.000,0.278,0.722,6395.0,6409.7,6520.0,6288.0,13159710.94,-0.01%,-0.01%,-0.02%,5.45,3.54,816889,546702.0,290227.0


In [376]:
print("Type of variables: ", "\n", df.dtypes)

Type of variables:  
 Time                        object
Favorites                  float64
Retweets                   float64
Compound                   float64
Negative                   float64
Positive                   float64
Neutral                    float64
Open                       float64
Close                      float64
High                       float64
Low                        float64
Volume                     float64
Funding Rate 0000           object
Funding Rate 0800           object
Funding Rate 1600           object
Bitcoin_SVI                float64
Cryptocurrency_SVI         float64
No. of Active Addresses      int64
No. of Unique Addresses    float64
Transactions               float64
dtype: object


# EDA & Preprocessing

In [377]:
# Select features, the target output is'Close'
feature_pool = ['Favorites','Retweets','Compound','Negative','Positive',\
                'Neutral','Open', 'High','Low',\
                'Volume','Funding Rate 0000','Funding Rate 0800','Funding Rate 1600','Bitcoin_SVI',\
                'Cryptocurrency_SVI','No. of Active Addresses','No. of Unique Addresses','Transactions']

output = 'Close'

## Data Type Transformation

In [378]:
# Transform object type of "Funding Rate 0000", "Funding Rate 0800" and " Funding Rate 1600" to numerical type
df['Funding Rate 0000'] = df['Funding Rate 0000'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))
df['Funding Rate 0800'] = df['Funding Rate 0800'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))
df['Funding Rate 1600'] = df['Funding Rate 1600'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))

## Train-Test Split

In [379]:
X, y = df.drop(output, axis=1), df[output]

# Split the data: train set: test set using = 70/30. Set random_state=42 so that results could be replicated.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
print(y_train.sum()/y_train.count(),y_test.sum()/y_test.count())

31054.29138943249 34150.09908675799


## Handle Missing Data

In [380]:
# Count missing value
print(df.isna().sum())

# check the consistency of columns with missing value between train set and the whole set
print(df[feature_pool].isnull().any().tolist()==X_train.isnull().any().tolist())
print(df[output].isnull().any().tolist()==y_train.isnull().any().tolist())

Time                         0
Favorites                  458
Retweets                   458
Compound                   458
Negative                   458
Positive                   458
Neutral                    458
Open                         0
Close                        0
High                         0
Low                          0
Volume                       0
Funding Rate 0000            0
Funding Rate 0800            0
Funding Rate 1600            0
Bitcoin_SVI                  0
Cryptocurrency_SVI           0
No. of Active Addresses      0
No. of Unique Addresses      2
Transactions                 0
dtype: int64
False
True


### Monthly Sentiment Analysis

In [381]:
tweets.insert(1, "Month", "NaN")

tweets['Month'] = tweets['Time'].str[:7]
tweets.head(10)

,Time,Month,Favorites,Retweets,Compound,Negative,Positive,Neutral
0,2020-03-12,2020-03,5.0,2.0,0.0772,0.091,0.100,0.809
1,2020-03-17,2020-03,98.0,96.0,0.4019,0.000,0.114,0.886
2,2020-03-20,2020-03,125.0,95.0,0.4767,0.000,0.146,0.854
3,2020-03-30,2020-03,3.0,0.0,0.0803,0.061,0.071,0.869
4,2020-03-31,2020-03,186.0,142.0,0.6597,0.000,0.278,0.722
5,2020-04-01,2020-04,171.0,120.0,0.5267,0.061,0.184,0.754
6,2020-04-16,2020-04,419.0,94.0,0.0000,0.000,0.000,1.000
7,2020-04-22,2020-04,73.0,18.0,0.3182,0.000,0.084,0.916
8,2020-04-30,2020-04,994.0,153.0,0.0000,0.000,0.000,1.000
9,2020-05-09,2020-05,35.0,2.0,0.0000,0.000,0.000,1.000


In [382]:
tweets = tweets.groupby(by="Month").mean()
tweets = tweets.reset_index()

In [383]:
X_train.insert(1, "Month", "NaN")
X_train['Month'] = X_train['Time'].str[:7]

X_test.insert(1, "Month", "NaN")
X_test['Month'] = X_test['Time'].str[:7]

In [384]:
df_X_train = pd.DataFrame(columns = ['Time', 'Month', 'Favorites', 'Retweets', 'Compound', \
                                 'Negative', 'Positive', 'Neutral', 'Open', 'High', \
                                 'Low', 'Volume', 'Funding Rate 0000', 'Funding Rate 0800', \
                                 'Funding Rate 1600', 'Bitcoin_SVI', 'Cryptocurrency_SVI',  'No. of Active Addresses', \
                                 'No. of Unique Addresses', 'Transactions'])

for i, row in X_train.iterrows():
    if (pd.isna(X_train.loc[i].at["Favorites"])):
        data = tweets[tweets['Month'] == X_train.loc[i].at["Month"]]
        new_row = {'Time': X_train.loc[i].at["Time"], 'Month': X_train.loc[i].at["Month"], 'Favorites': data.Favorites.values[0], 'Retweets': data.Retweets.values[0], 'Compound': data.Compound.values[0], \
                   'Negative': data.Negative.values[0], 'Positive': data.Positive.values[0], 'Neutral': data.Neutral.values[0], 'Open': X_train.loc[i].at["Open"], \
                   'High': X_train.loc[i].at["High"], 'Low': X_train.loc[i].at["Low"], 'Volume': X_train.loc[i].at["Volume"], 'Funding Rate 0000': X_train.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_train.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_train.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_train.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_train.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_train.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_train.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_train.loc[i].at["Transactions"]}
        df_X_train = df_X_train.append(new_row, ignore_index=True)
    else:
        new_row = {'Time': X_train.loc[i].at["Time"], 'Month': X_train.loc[i].at["Month"], 'Favorites': X_train.loc[i].at["Favorites"], 'Retweets': X_train.loc[i].at["Retweets"], 'Compound': X_train.loc[i].at["Compound"], \
                   'Negative': X_train.loc[i].at["Negative"], 'Positive': X_train.loc[i].at["Positive"], 'Neutral': X_train.loc[i].at["Neutral"], 'Open': X_train.loc[i].at["Open"],\
                   'High': X_train.loc[i].at["High"], 'Low': X_train.loc[i].at["Low"], 'Volume': X_train.loc[i].at["Volume"], 'Funding Rate 0000': X_train.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_train.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_train.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_train.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_train.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_train.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_train.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_train.loc[i].at["Transactions"]}
        df_X_train = df_X_train.append(new_row, ignore_index=True)


In [385]:
df_X_test = pd.DataFrame(columns = ['Time', 'Month', 'Favorites', 'Retweets', 'Compound', \
                                 'Negative', 'Positive', 'Neutral', 'Open', 'High', \
                                 'Low', 'Volume', 'Funding Rate 0000', 'Funding Rate 0800', \
                                 'Funding Rate 1600', 'Bitcoin_SVI', 'Cryptocurrency_SVI',  'No. of Active Addresses', \
                                 'No. of Unique Addresses', 'Transactions'])

for i, row in X_test.iterrows():
    if (pd.isna(X_test.loc[i].at["Favorites"])):
        data = tweets[tweets['Month'] == X_test.loc[i].at["Month"]]
        new_row = {'Time': X_test.loc[i].at["Time"], 'Month': X_test.loc[i].at["Month"], 'Favorites': data.Favorites.values[0], 'Retweets': data.Retweets.values[0], 'Compound': data.Compound.values[0], \
                   'Negative': data.Negative.values[0], 'Positive': data.Positive.values[0], 'Neutral': data.Neutral.values[0], 'Open': X_test.loc[i].at["Open"], \
                   'High': X_test.loc[i].at["High"], 'Low': X_test.loc[i].at["Low"], 'Volume': X_test.loc[i].at["Volume"], 'Funding Rate 0000': X_test.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_test.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_test.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_test.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_test.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_test.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_test.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_test.loc[i].at["Transactions"]}
        df_X_test = df_X_test.append(new_row, ignore_index=True)
    else:
        new_row = {'Time': X_test.loc[i].at["Time"], 'Month': X_test.loc[i].at["Month"], 'Favorites': X_test.loc[i].at["Favorites"], 'Retweets': X_test.loc[i].at["Retweets"], 'Compound': X_test.loc[i].at["Compound"], \
                   'Negative': X_test.loc[i].at["Negative"], 'Positive': X_test.loc[i].at["Positive"], 'Neutral': X_test.loc[i].at["Neutral"], 'Open': X_test.loc[i].at["Open"], \
                   'High': X_test.loc[i].at["High"], 'Low': X_test.loc[i].at["Low"], 'Volume': X_test.loc[i].at["Volume"], 'Funding Rate 0000': X_test.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_test.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_test.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_test.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_test.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_test.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_test.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_test.loc[i].at["Transactions"]}
        df_X_test = df_X_test.append(new_row, ignore_index=True)


In [386]:
X_test = df_X_test
X_test = X_test.drop(['Time'], axis=1)
X_test = X_test.drop(['Month'], axis=1)
X_test

,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,1769.357143,184.857143,0.241271,0.002214,0.134714,0.863143,40578.8,41400.0,38734.2,1.370817e+08,0.13,0.06,0.07,26.45,17.35,1094947,760179.0,321389.0
1,1133.000000,112.000000,0.410050,0.000000,0.123500,0.876500,43819.1,48466.6,43293.6,3.634166e+08,0.01,0.00,0.00,11.94,19.83,962201,697433.0,277042.0
2,1870.122024,278.058036,0.376858,0.009379,0.122107,0.868536,41018.9,42593.0,40752.1,1.849269e+08,0.01,0.00,0.00,9.81,17.97,830989,589252.0,241991.0
3,1111.875000,148.500000,0.328937,0.067875,0.160125,0.772000,32283.7,34752.4,32001.2,2.055358e+08,-0.01,-0.02,-0.05,12.96,17.35,518219,379066.0,124640.0
4,254.666667,37.833333,0.117625,0.049250,0.123000,0.827750,15686.5,16338.3,15450.0,3.791161e+07,0.01,0.01,0.01,8.51,6.20,1064270,721180.0,324855.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,2113.750000,253.750000,0.293231,0.026500,0.124750,0.848688,34256.7,35400.0,33848.3,1.419700e+08,-0.01,-0.03,-0.02,14.04,17.97,641796,440908.0,174535.0
215,300.000000,50.000000,0.405267,0.000000,0.146000,0.854000,12785.0,13183.0,12678.5,4.897670e+07,0.01,0.01,0.01,14.26,6.40,1048775,753892.0,339087.0
216,63.500000,17.000000,0.121117,0.000000,0.035000,0.965000,10528.8,10535.4,10137.6,3.168681e+07,0.01,0.00,0.00,4.71,4.78,1013565,733591.0,349938.0
217,407.000000,21.000000,0.000000,0.000000,0.000000,1.000000,47108.1,48444.2,46693.0,2.182019e+08,0.01,0.01,0.01,10.26,21.08,976201,654103.0,268957.0


In [387]:
X_train = df_X_train
X_train = X_train.drop(['Time'], axis=1)
X_train = X_train.drop(['Month'], axis=1)
X_train

,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,414.250000,96.250000,0.211225,0.015250,0.067000,0.917500,7692.9,7793.0,7624.5,1.439023e+07,0.01,0.01,0.01,7.60,4.25,876229,629817.0,311217.0
1,232.000000,155.000000,0.401900,0.000000,0.072000,0.928000,9280.7,9554.7,9238.4,1.728008e+07,0.01,0.01,0.00,5.94,4.07,909615,635165.0,318274.0
2,1417.333333,237.000000,0.332800,0.000000,0.107333,0.892667,63221.9,63506.8,60400.0,4.504493e+08,0.05,0.08,0.04,11.70,48.33,1069212,724143.0,288746.0
3,1111.875000,148.500000,0.328937,0.067875,0.160125,0.772000,35479.6,36191.3,33335.0,2.471069e+08,0.01,0.01,0.01,17.28,22.92,652470,441204.0,167249.0
4,232.000000,155.000000,0.401900,0.000000,0.072000,0.928000,9464.6,9484.6,9278.4,1.432157e+07,0.01,-0.01,-0.01,5.24,3.72,854866,582343.0,283897.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,4111.666667,334.666667,0.291200,0.000000,0.100000,0.900000,57630.8,59351.9,57072.8,2.081187e+08,0.02,0.03,0.03,19.87,26.36,1154381,826466.0,321090.0
507,400.000000,33.000000,0.911700,0.000000,0.249000,0.751000,34703.8,35291.5,33857.4,2.220320e+08,-0.03,0.00,0.00,14.09,18.31,776365,580308.0,217898.0
508,534.200000,90.366667,0.068437,0.037100,0.068967,0.893967,44428.2,45367.5,43332.8,3.404605e+08,0.00,0.00,0.01,15.61,26.33,1091796,731603.0,288657.0
509,74.400000,21.400000,0.070720,0.063800,0.078800,0.857400,57420.6,57517.8,44160.0,3.305140e+08,0.13,0.12,0.09,33.74,24.61,1094544,761941.0,285383.0


### Unique Addresses

In [388]:
KNN_miss_filling = KNNImputer(n_neighbors=2).fit(X_train)
X_train = pd.DataFrame(KNN_miss_filling.transform(X_train), columns=feature_pool)
X_test = pd.DataFrame(KNN_miss_filling.transform(X_test), columns=feature_pool)

In [389]:
print(X_train.isna().sum())
print(X_test.isna().sum())

Favorites                  0
Retweets                   0
Compound                   0
Negative                   0
Positive                   0
Neutral                    0
Open                       0
High                       0
Low                        0
Volume                     0
Funding Rate 0000          0
Funding Rate 0800          0
Funding Rate 1600          0
Bitcoin_SVI                0
Cryptocurrency_SVI         0
No. of Active Addresses    0
No. of Unique Addresses    0
Transactions               0
dtype: int64
Favorites                  0
Retweets                   0
Compound                   0
Negative                   0
Positive                   0
Neutral                    0
Open                       0
High                       0
Low                        0
Volume                     0
Funding Rate 0000          0
Funding Rate 0800          0
Funding Rate 1600          0
Bitcoin_SVI                0
Cryptocurrency_SVI         0
No. of Active Addresses    0
N